In [1]:
%matplotlib inline
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from preprocessing import *
import cv2


from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Lambda
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

import time


# to autoreload modules on code change
%load_ext autoreload
%autoreload 2

/home/u8710/.conda/envs/my_root/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
datagen = ImageDataGenerator(
#         featurewise_center=True,
#         featurewise_std_normalization=True,
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
        rescale=1./255,
#         shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
#         preprocessing_function= lambda x: cv2.resize(x,(150,150))
)

test_datagen = ImageDataGenerator(
        rescale=1./255,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
#         preprocessing_function= lambda x: cv2.resize(x,(150,150))
)


In [3]:
trn_path = './data/train/'
val_path = './data/val/'
bs = 32


dl = datagen.flow_from_directory(trn_path,
                                 target_size=(200,200),
                                 class_mode='categorical',
                                 batch_size=32,
                                 shuffle=True)

test_dl = datagen.flow_from_directory(val_path,
                       target_size=(200,200),
                       class_mode='categorical',
                       batch_size=32,
                       shuffle=True)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [4]:
n_tr = 28709
n_val = 3589

In [5]:

shp = (200,200,3)
model = ResNet50(weights='imagenet',include_top=False,input_shape=shp)

In [6]:
for layer in model.layers:
    layer.trainable = False

In [7]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(7, activation="softmax")(x)

In [8]:
model2 = Model(inputs = model.input, outputs = predictions)

model2.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [9]:
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 100, 100, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 100, 100, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100, 100, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

In [10]:
def sclr(x,y):
    return y*0.95

In [11]:
fp = './models/weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(fp, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
# lr_s = LearningRateScheduler(sclr, verbose=1)

In [12]:
model2.fit_generator(dl,
                    steps_per_epoch = n_tr/bs,
                    epochs = 30,
                    validation_data = test_dl,
                    validation_steps = 10,
                    use_multiprocessing=True,
                    callbacks = [checkpoint, early])

Epoch 1/30
897/897 [============================>.] - ETA: 0s - loss: 1.6666 - acc: 0.3440
Epoch 00001: val_acc improved from -inf to 0.18125, saving model to ./models/weights.01-1.85.hdf5
898/897 [==============================] - 2592s 3s/step - loss: 1.6664 - acc: 0.3439 - val_loss: 1.8513 - val_acc: 0.1812
Epoch 2/30
897/897 [============================>.] - ETA: 0s - loss: 1.5654 - acc: 0.3888
Epoch 00002: val_acc did not improve
898/897 [==============================] - 2573s 3s/step - loss: 1.5654 - acc: 0.3887 - val_loss: 2.1685 - val_acc: 0.1406
Epoch 3/30
897/897 [============================>.] - ETA: 0s - loss: 1.5288 - acc: 0.4073
Epoch 00003: val_acc improved from 0.18125 to 0.18438, saving model to ./models/weights.03-2.26.hdf5
898/897 [==============================] - 2590s 3s/step - loss: 1.5287 - acc: 0.4074 - val_loss: 2.2583 - val_acc: 0.1844
Epoch 4/30
141/897 [===>..........................] - ETA: 35:36 - loss: 1.5251 - acc: 0.4047

Process ForkPoolWorker-1:
Process ForkPoolWorker-5:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/queues.py", line 335, in get
    res = self._reader.recv_bytes()
  File "/home/u8710/.conda/envs/my_root/lib/python3.6/multiprocessing/pool.py", line 108, in

KeyboardInterrupt: 